# Algorithm & Metric Deep Dive - Making appropriate choices for your problem

NOOA template sections
## Overview 



### Prerequisites
xx


### Learning Outcomes 
xx

## Tutorial 

Description of decision tree
* basic tree
* training
* measuring performance
* advanced trees - random forest
* advanced gradient boosted trees

Description of NNs

Training
* back propogation
* mini batches

Types of NN
* feed forward
* CNN
* LSTM
* Graph NN

Description


## Excercise 
xx


## Problem 3 - Radiation Emulation (Regression)

## Examples of use
xx


## Next steps
xx


## Dataset Info
xx

## References
xx
